## Agile Hardware Design
***
# Queue Design Case Study

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Spring21/)

## Plan for Today

* Designing for reuse
* Designing a Queue
* Iteratively improving Queue design

## Loading The Chisel Library Into a Notebook

In [1]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/Users/sbeamer/Spring 2021/CSE 293/lectures/13-queue/../resource/chisel_deps.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

## Goals for Reuse

* Need to recognize _pattern_ of functionality

* Include necessary parameterization and generation to support users' needs

## Planning for Progressive Design

* Reduce the complexity/challenge of any one step
* _Close the loop_ as early as possible, then augment/extend/revise
* Look for opportunities to defer features/optimizations to later
* While developing, re-evaluate plan and revise as needed
* Consider
  * What is the simplest thing I can implement?
  * How can test it? (both at start and as it evolves)
  * Plotting a roadmap of order to develop features/optimizations

## Case Study: Designing a Queue

* A _Queue_ with `Decoupled` interfaces on both sides
  * Would like to parameterize queue depth and data types
  * Power/performance/area (PPA) goals

* Deferred features
  * Parameters (queue depth & data type)
  * Performance (correct but slow ok at first)

<img src="images/queue.svg" alt="queue high-level" style="width:70%;margin-left:auto;margin-right:auto"/>

## First Attempt at Queue

* _Simplification_: only single entry

<img src="images/single.svg" alt="single-entry queue" style="width:60%;margin-left:auto;margin-right:auto"/>

## V0 - First Attempt at Queue

In [130]:
class MyQueue(bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    val entry = Reg(UInt(bitWidth.W))
    val full = RegInit(false.B)
    io.enq.ready := !full || io.deq.fire
    io.deq.valid := full
    io.deq.bits := entry
    when (io.deq.fire) {
        full := false.B
    }
    when (io.enq.fire) {
        entry := io.enq.bits
        full := true.B
    }
}

defined class MyQueue

## Testing Our Queue - Scala Model

In [53]:
class QueueModel(numEntries: Int) {
    val mq = scala.collection.mutable.Queue[Int]()
    var deqReady = false

    def attemptEnq(elem: Int) { 
        if (enqReady()) mq += elem
    }

    // call first within a cycle
    // improve with Option & None
    def attemptDeq() = if (deqReady) mq.dequeue() else -1
    
    def enqReady() = mq.size < numEntries || (mq.size == numEntries && deqReady)
    def deqValid() = mq.nonEmpty
}

defined class QueueModel

## Testing Our Queue - Harness + Simulation

In [142]:
def simCycle(qm: QueueModel, c: MyQueue, enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    if (c.io.deq.valid.peek.litToBoolean && deqReady) {
        assert(qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    }
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

defined function simCycle

## Testing Our Queue - Simulation

In [136]:
test(new MyQueue(8)) { c =>
    val qm = new QueueModel(1)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, true, false, 2)
    simCycle(qm, c, true, true, 2)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1)
Queue(2)
Queue()
test MyQueue Success: 0 tests passed in 7 cycles in 0.003292 seconds 2126.53 Hz


## Assessing `MyQueue V0`

* Accomplished
    * Implements queueing behavior
    * Parameterized data width
* Shortcommings
    * Only one entry (_next goal_ to fix)

## Parameterizing Number of Queue Entries

* First attempt at parameterizing number of entries: _shift register_

<img src="images/shift.svg" alt="queue via shift register" style="width:60%;margin-left:auto;margin-right:auto"/>

## V1 - Parameterizing Number of Queue Entries

In [138]:
class MyQueue(numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 0)
    val entries = Seq.fill(numEntries)(Reg(UInt(bitWidth.W)))
    val fullBits = Seq.fill(numEntries)(RegInit(false.B))
    val shiftDown = io.deq.fire || !fullBits.head
    io.enq.ready := !fullBits.last || shiftDown
    io.deq.valid := fullBits.head
    io.deq.bits := entries.head
    when (shiftDown) { // dequeue / shift
        for (i <- 0 until numEntries - 1) {
            entries(i) := entries(i+1)
            fullBits(i) := fullBits(i+1)
        }
        fullBits.last := false.B
    }
    when (io.enq.fire) { // enqueue
        entries.last := io.enq.bits
        fullBits.last := true.B
    }
//     when (shiftDown || io.enq.fire) {
//         entries.foldRight(io.enq.bits){(thisEntry, lastEntry) => thisEntry := lastEntry; thisEntry}
//         fullBits.foldRight(io.enq.fire){(thisEntry, lastEntry) => thisEntry := lastEntry; thisEntry}
//     }
}

defined class MyQueue

## Testing Our Queue

In [140]:
test(new MyQueue(2,8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, false, true)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1)
Queue()
test MyQueue Success: 0 tests passed in 6 cycles in 0.004447 seconds 1349.08 Hz


## Assessing `MyQueueV1`

* Accomplished
    * Implements queueing behavior
    * Parameterized data width & _number of entries_
* Shortcommings
    * Long latency when queue is empty (all elements go through all entries)
    * Not good at handling bubbles midway (might even be buggy)

## Squishing Bubbles in Queue

* Use a _Priority Encoder_ to squeeze out bubbles
  * Insert in first free slot

<img src="images/shift.svg" alt="queue via shift register" style="width:60%;margin-left:auto;margin-right:auto"/>

## V2 - Using Priority Encoder for Insertion

In [141]:
class MyQueue(numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 0)
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val fullBits = RegInit(VecInit(Seq.fill(numEntries)(false.B)))
    val emptyBits = fullBits map { !_ }
    io.enq.ready := emptyBits reduce { _ || _ } // any empties?
    io.deq.valid := fullBits.head
    io.deq.bits := entries.head
    when (io.deq.fire) { // dequeue & shift up
        for (i <- 0 until numEntries - 1) {
            entries(i) := entries(i+1)
            fullBits(i) := fullBits(i+1)
        }
        fullBits(numEntries - 1) := false.B
    }
    when (io.enq.fire) { // priority enqueue
        val writeIndex = PriorityEncoder(emptyBits)
        entries(writeIndex) := io.enq.bits
        fullBits(writeIndex) := true.B
    }
}

defined class MyQueue

## Testing Our Queue

In [143]:
test(new MyQueue(2, 8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue()
test MyQueue Success: 0 tests passed in 5 cycles in 0.002772 seconds 1803.85 Hz


## Assessing `MyQueue V2`

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy

* Shortcommings
  * _Performance:_ can't simultaneously enqueue/dequeue to a full queue
  * _Power Efficiency:_ lots of bits shifting
  * _Potential Critical Path:_ priority encoder logic depth

## Keeping Data in Place with a Circular Buffer

* _Circular buffer_ uses two pointers (indices) and fixed size storage to make a FIFO
  * Insert new data at _in_ (and increment _in_)
  * Pop from _out_ (and increment _out_)
  * Wrap pointers around when they get to end
* How to tell when empty vs full?
  * First try: _empty_ when pointers are equal, _full_ when in+1 == out

<img src="images/circular.svg" alt="circular buffer" style="width:60%;margin-left:auto;margin-right:auto"/>

## V3 - Keeping Data in Place with Circular Buffer

In [145]:
class MyQueue(numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W))) // Mem?
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val empty = enqIndex === deqIndex
    val full = (enqIndex +% 1.U) === deqIndex
    io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
    }
}

defined class MyQueue

In [151]:
def simCycle(qm: QueueModel, c: MyQueue, enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    if (deqReady && qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueue(2, 8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
test MyQueue Success: 0 tests passed in 3 cycles in 0.001812 seconds 1655.18 Hz


: 

## Assessing MyQueueV3

* Accomplished
  * Implements queueing behavior
  * Parameterized data width & number of entries
  * Latency based on occupancy
  * Efficiency? Less bits shifting and shallower logic

* Shortcommings
  * _Capacity:_ loose one entry (and must be power of 2)
  * _Performance:_ can't simultaneously enqueue/dequeue to a full queue

## V4 - Adding State (`maybeFull`) Track Last Entry

In [152]:
class MyQueue(numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val maybeFull = RegInit(false.B)
    val empty = enqIndex === deqIndex && !maybeFull
    val full = enqIndex === deqIndex && maybeFull
    io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
        when (enqIndex =/= deqIndex) {
            maybeFull := false.B
        }
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
        when ((enqIndex +% 1.U) === deqIndex) {
            maybeFull := true.B
        }
    }
}

defined class MyQueue

In [104]:
def simCycle(qm: QueueModel, c: MyQueueV4, enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    if (deqReady && qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueueV4(2, 8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue()
test MyQueueV4 Success: 0 tests passed in 5 cycles in 0.004057 seconds 1232.38 Hz


defined function simCycle

## V5 - Simultaneous Enqueue/Dequeue When Full

In [153]:
class MyQueue(numEntries: Int, bitWidth: Int) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val enqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val deqIndex = RegInit(0.U(log2Ceil(numEntries).W))
    val maybeFull = RegInit(false.B)
    val empty = enqIndex === deqIndex && !maybeFull
    val full = enqIndex === deqIndex && maybeFull
    io.enq.ready := !full || io.deq.ready  // NOTE: io.enq.ready now attached to io.deq.ready
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex)
    when (io.deq.fire) {
        deqIndex := deqIndex +% 1.U
        when (enqIndex =/= deqIndex) {
            maybeFull := false.B
        }
    }
    when (io.enq.fire) {
        entries(enqIndex) := io.enq.bits
        enqIndex := enqIndex +% 1.U
        when ((enqIndex +% 1.U) === deqIndex) {
            maybeFull := true.B
        }
    }
}

defined class MyQueue

In [154]:
def simCycle(qm: QueueModel, c: MyQueue, enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    if (deqReady && qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueue(2, 8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, true, false, 2)
    simCycle(qm, c, true, true, 3)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(2, 3)
Queue(3)
test MyQueue Success: 0 tests passed in 7 cycles in 0.004083 seconds 1714.23 Hz


defined function simCycle

## V6 - Tidying up Code

In [155]:
class MyQueue(numEntries: Int, bitWidth: Int, pipe: Boolean=true) extends Module {
    val io = IO(new Bundle {
        val enq = Flipped(Decoupled(UInt(bitWidth.W)))
        val deq = Decoupled(UInt(bitWidth.W))
    })
    require(numEntries > 1)
    require(isPow2(numEntries))
    val entries = Reg(Vec(numEntries, UInt(bitWidth.W)))
    val enqIndex = Counter(numEntries)
    val deqIndex = Counter(numEntries)
    val maybeFull = RegInit(false.B)
    val indicesEqual = enqIndex.value === deqIndex.value
    val empty = indicesEqual && !maybeFull
    val full = indicesEqual && maybeFull
    if (pipe)
        io.enq.ready := !full || io.deq.ready
    else
        io.enq.ready := !full
    io.deq.valid := !empty
    io.deq.bits := entries(deqIndex.value)
    when (indicesEqual && io.deq.fire =/= io.enq.fire) {
        maybeFull := !maybeFull
    }
    when (io.deq.fire) {
        deqIndex.inc()
    }
    when (io.enq.fire) {
        entries(enqIndex.value) := io.enq.bits
        enqIndex.inc()
    }
}

defined class MyQueue

In [156]:
def simCycle(qm: QueueModel, c: MyQueue, enqReady: Boolean, deqReady: Boolean, enqData: Int=0) {
    qm.deqReady = deqReady
    c.io.deq.ready.poke(qm.deqReady.B)
    c.io.deq.valid.expect(qm.deqValid.B)
    if (deqReady && qm.deqValid)
        c.io.deq.bits.expect(qm.attemptDeq().U)
    c.io.enq.ready.expect(qm.enqReady.B)
    c.io.enq.valid.poke(enqReady.B)
    c.io.enq.bits.poke(enqData.U)
    if (enqReady)
        qm.attemptEnq(enqData)
    c.clock.step()
    println(qm.mq)
}

test(new MyQueue(2, 8)) { c =>
    val qm = new QueueModel(2)
    simCycle(qm, c, false, false)
    simCycle(qm, c, true, false, 1)
    simCycle(qm, c, true, false, 2)
    simCycle(qm, c, true, true, 3)
    simCycle(qm, c, false, true)
}

Elaborating design...
Done elaborating.
Queue()
Queue(1)
Queue(1, 2)
Queue(2, 3)
Queue(3)
test MyQueue Success: 0 tests passed in 7 cycles in 0.004239 seconds 1651.15 Hz


defined function simCycle